In [1]:
import requests
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pandas as pd
import numpy as np

In [31]:
def fetch_books_data(query_params):
    """
    Fetch raw data from the Google Books API with flexible query parameters.

    :param query_params: A dictionary of query parameters (e.g., {"intitle": "Python", "inauthor": "Guido"}).
    :return: The raw JSON response from the API.
    """
    # Define the base URL for the Google Books API
    url = "https://www.googleapis.com/books/v1/volumes"

    # Dynamically build the query string from the dictionary of query parameters
    # Each key-value pair in the dictionary is formatted as "key:value"
    # The "join" function combines these pairs with spaces to create the full query string
    query = " ".join(f"{key}:{value}" for key, value in query_params.items())

    # Define the parameters for the API request
    params = {
        "q": query,  # The query string constructed above
        "maxResults": 13  # Limit the number of results returned by the API to 10
    }

    try:
        # Send a GET request to the Google Books API with the specified parameters
        response = requests.get(url, params=params)

        # Check for HTTP errors; raise an exception if the response status indicates an error
        response.raise_for_status()

        # If the request is successful, return the JSON data from the API response
        return response.json()

    except requests.exceptions.RequestException as e:
        # If an error occurs during the request, print the error message
        print(f"API request error: {e}")

        # Return an empty dictionary to signify failure
        return {}


In [3]:
def format_json(data):
    """
    Format and pretty-print JSON data.

    :param data: The JSON data (as a Python dictionary).
    :return: A formatted string representation of the JSON data.
    """
    try:
        # Convert JSON data to a pretty-printed string
        return json.dumps(data, indent=4)
    except (TypeError, ValueError) as e:
        # Handle errors in case the input is not valid JSON
        print(f"Error formatting JSON: {e}")
        return "{}"



In [4]:
def json_to_dataframe(raw_data):
    """
    Transform JSON data from the Google Books API into a pandas DataFrame.

    :param json_data: Raw JSON data from the Google Books API.
    :return: A pandas DataFrame containing relevant book information.
    """
    try:
        # Extract the list of books from the JSON data
        items = json_data.get("items", [])

        # Prepare a list to store extracted metadata
        metadata = []

        # Iterate through the books and extract relevant fields
        for item in items:
            volume_info = item.get("volumeInfo", {})
            record = {
                "ID": item.get("id", np.nan),
                "Title": volume_info.get("title", np.nan),
                "Authors": ", ".join(volume_info.get("authors", [])) if "authors" in volume_info else np.nan, #note this may return an emtpy [] not NAN
                "Publisher": volume_info.get("publisher", np.nan),
                "Page Count": volume_info.get("pageCount", np.nan),
                "Language": volume_info.get("language", np.nan),
                "Category": ", ".join(volume_info.get("categories", [])) if "categories" in volume_info else np.nan, #note this may return an emtpy [] not NAN
                "Thumbnail": volume_info.get("imageLinks", {}).get("thumbnail", np.nan), #note this may return an emtpy {} not NAN


                "ISBN_13": next(
                    (
                        identifier["identifier"]
                        for identifier in volume_info.get("industryIdentifiers", [])
                        if identifier["type"] == "ISBN_13"
                    ),
                    np.nan,
                ),
                "ISBN_10": next(
                    (
                        identifier["identifier"]
                        for identifier in volume_info.get("industryIdentifiers", [])
                        if identifier["type"] == "ISBN_10"
                    ),
                    np.nan,
                ),
            }
            metadata.append(record)

        # Create a DataFrame from the metadata
        return pd.DataFrame(metadata)

    except Exception as e:
        print(f"Error processing JSON data: {e}")
        return pd.DataFrame()


In [36]:
def select_book_from_results(books_df):
    """
    Allows the user to select a book from the results in the DataFrame,
    displaying them in batches of 5. If '6' is selected, the next batch is shown.

    :param books_df: Pandas DataFrame containing book data.
    :return: A dictionary of the selected book's metadata or None if no valid selection is made.
    """
    # Check if the DataFrame is empty
    if books_df.empty:
        print("No books available to select.")
        return None  # Can be enhanced by asking to repeat

    start_index = 0  # Initial starting index for displaying books
    print(f"There were " + str(len(books_df)) + " books found")

    while True:
        # Display the next batch of books (5 at a time)
        end_index = start_index + 5
        subset_books = books_df.iloc[start_index:end_index]

        if subset_books.empty:
            print("No more books to display.")
            return None  # Stop if there are no more books to display

        print("\nAvailable books:")
        for i, row in subset_books.iterrows():
            print(f"{i + 1}: {row['Title']} by {row['Authors']}")

        if int(len(subset_books)) % 5 == 0:
            print("6: Show the next 5 books (if available)")

        print("0: Cancel selection")

        # Prompt the user to choose a book
        try:
            choice = int(input(f"\nEnter the number of the book you want to select (1-5), 6 to show more, or 0 to cancel: "))
            if 1 <= choice <= len(subset_books):
                # Return the selected book's data as a dictionary
                selected_book = subset_books.iloc[choice - 1 - start_index].to_dict()
                print(f"\nYou selected: {selected_book['Title']} by {selected_book['Authors']}")
                return selected_book
            elif choice == 6:
                # Show the next batch of books
                start_index += 5
            elif choice == 0:
                print("Selection canceled.")
                return None
            else:
                print("Invalid selection. Please choose a valid option.")
        except ValueError:
            print("Invalid input. Please enter a number between 1 and 6, or 0 to cancel.")


In [37]:
# Define your search query (e.g., by title or author)
# example of serach query:
#query_params = {"intitle": "Python", "inauthor": "Guido", "isbn": "9781449355739"}
query_params = {"intitle": "Python"}

raw_data = fetch_books_data(query_params)
book_df= json_to_dataframe(raw_data)
selection = select_book_from_results(book_df)
print(selection)
#formatted_json = format_json(raw_data)
#books_df = json_to_dataframe(raw_data)
#print(formatted_json)
#print(books_df.head())

There were 13 books found

Available books:
1: Programming in Python 3 by Mark Summerfield
2: Core Python Programming by Wesley Chun
3: Learning Python by Mark Lutz, David Ascher
4: Python Programming by John M. Zelle
5: Introduction to Computation and Programming Using Python, second edition by John V. Guttag
6: Show the next 5 books (if available)
0: Cancel selection

Enter the number of the book you want to select (1-5), 6 to show more, or 0 to cancel: 6

Available books:
6: Learn Python by Damon Parker
7: Python Cookbook by David Beazley, Brian K. Jones
8: Head First Python by Paul Barry
9: Python Crash Course, 3rd Edition by Eric Matthes
10: Core Python Applications Programming by Wesley J Chun
6: Show the next 5 books (if available)
0: Cancel selection

Enter the number of the book you want to select (1-5), 6 to show more, or 0 to cancel: 6

Available books:
11: Python Crash Course by Eric Matthes
12: Python Programming on Win32 by Mark J. Hammond, Andy Robinson
13: Natural Langu

In [ ]:
# Set display options
pd.set_option('display.max_colwidth', None)  # This ensures that full content is shown in each column
pd.set_option('display.max_rows', 100)      # Set the number of rows you want to display
pd.set_option('display.max_columns', 20)    # Set the number of columns you want to display
